In [2]:
#https://www.dummies.com/programming/python/plotting-a-sound-file-in-ipython/
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np
from gurobipy import *
from IPython.display import Audio
import os
import glob
import untangle

import xml.etree.ElementTree as et
import pandas as pd

# Functions

In [3]:
# Function for transforming a signal with fourier
def fourier_trans(data, rate, begin_time, end_time): 
    # initialising variables
    tp_count = len(data)
    values = np.arange(int(tp_count/2))
    time_period = tp_count/rate
    frequencies = values/time_period
    
    #at what intervals time points are sampled
    sampling_interval = 1/rate;
    
    #time points
    time = np.arange(begin_time, end_time, sampling_interval);
    
    #Frequency domain representation
    fourier_transform = fft(data)/tp_count #normalized the amplitude
    fourier_transform = fourier_transform[range(int(len(data)/2))] #exclude sampling frequency   
    return frequencies, abs(fourier_transform)

# TODO: Funktion überarbeiten damit man es auch versteht!
def new_fft(data, rate):
    N = len(data)

    # sample spacing
    T = 1.0 / rate
    x = np.linspace(0.0, N*T, N)

    yf = fft(data)
    freq = np.linspace(0.0, 1.0/(2.0*T), N//2)
    ampl = 2.0/N * np.abs(yf[0:N//2])
    return freq, ampl

# Frequency domain representation
def plot_freq_domain(freq, fourier, name):
    plt.figure(figsize=(20,5))
    plt.plot(freq, fourier)
    plt.xlabel('Frequency in Hz')
    plt.ylabel('Amplitude')
    plt.xlim(0, 1400)
    plt.title(name)
    plt.show()

# TODO: Warnung herausgeben wenn bei der Teilung der Samplerate eine Fliesskommazahl entsteht
def easy_downsampling(data, rate, sampl_fac):
    ds_data = []
    for i in range(len(data)):
        if (i%sampl_fac == 0):
            ds_data.append(data[i])
    ds_rate = int(rate/sampl_fac)
    return ds_data, ds_rate

#ToDo: startwert von 0 nicht möglich überarbeiten!!
def snip_wav(data, rate, start_sec, end_sec):
    start_point = int((rate * start_sec)-1)
    end_point = int((rate * end_sec)-1)
    #print('start_point:', start_point)
    #print('end_point:', end_point)
    snip_data = data[start_point:end_point]
    #print(snip_data)  
    return snip_data, rate

def metric(target_v, approx_v):
    norm_factor = np.linalg.norm(approx_v)
    if (norm_factor != 0):
        norm_v = approx_v/norm_factor
        diff = target_v - norm_v
        diff_sum = np.sum(np.abs(diff))
        return diff_sum
    else:
        print('Der Vektor kann nicht normalisiert werden.')
        return approx_v 

##  Set audio snippets and sample compression

In [9]:
start_sec = 1
end_sec = 2.5
sampl_fac = 4

# Dictonary

In [ ]:
## NEW
## Read WAV File
#
#audio_files = []
#path_wav = "audiofile"
#path_xml = "annotation"
#
## Init dict (combining same pitches into one)
#ref_note = {}
#note = 0
#
#for filename in sorted(glob.glob(os.path.join(path_xml, '*.xml'))):
#    
#    # Read data from xml
#    obj = untangle.parse(filename)
#    audiofile = obj.instrumentRecording.globalParameter.audioFileName.cdata
#    pitch = obj.instrumentRecording.transcription.event.pitch.cdata
#    #onsetSec = obj.instrumentRecording.transcription.event.onsetSec.cdata
#    #offsetSec = obj.instrumentRecording.transcription.event.offsetSec.cdata
#    fretNumber = obj.instrumentRecording.transcription.event.fretNumber.cdata
#    stringNumber = obj.instrumentRecording.transcription.event.stringNumber.cdata
#    
#    # Read wav
#    wav_file = path_wav + '/' + audiofile
#    rate, data = wav.read(wav_file)
#    
#    # Snip and downsampling audio-file
#    data, rate = snip_wav(data, rate, start_sec, end_sec)
#    data, rate = easy_downsampling(data, rate, sampl_fac)
#    
#    # Calc FFT
#    #freq, fourier = fourier_trans(data, rate, float(onsetSec), float(offsetSec))
#    #freq, fourier = fourier_trans(data, rate, 0, 2.5)
#    #freq, fourier = fourier_trans(data, rate, 0, end_sec-start_sec)
#    freq, fourier = new_fft(data, rate)
#    
#    # Update Dict
#    #ref_note[filename] = {
#    ref_note[note] = {
#        "pitch": int(pitch),
#        "fret": int(fretNumber),
#        "string": int(stringNumber),
#        "rate": int(rate),
#        "data": data,
#        #"onset": float(onsetSec), 
#        #"offset": float(offsetSec), 
#        "freq": freq,
#        "fourier": fourier,
#    }
#    note+=1

## Dataframe 

In [6]:
#xml_file = 'annotation/G53-40100-1111-00001.xml'
xml_file = 'recordings/annotation/instrumental_lead.xml'
tree = et.parse(xml_file)
root = tree.getroot()

In [7]:
df_cols = ['audioFileName', 'pitch', 'onsetSec', 'fretNumber']
rows = []

for globalParam in root.findall('globalParameter'):
    file_name = globalParam.find('audioFileName').text

for transcription in root.findall('transcription'):
    #print('tran:',transcription)
    
    for event in transcription.findall('event'):
        #print('event:', event)
        res = []
        res.append(file_name)
        
        for elem in df_cols[1:]:
            if event is not None and event.find(elem) is not None:
                res.append(event.find(elem).text)
            else:
                res.append(None)
        
        rows.append({df_cols[i]: res[i] for i, _ in enumerate(df_cols)})
        #print('rows', rows)

    out_df = pd.DataFrame(rows, columns=df_cols)
out_df

,audioFileName,pitch,onsetSec,fretNumber
0,\instrumental_lead.wav,64,0.519,14
1,\instrumental_lead.wav,66,0.78,16
2,\instrumental_lead.wav,67,0.968,17
3,\instrumental_lead.wav,64,1.179,14
4,\instrumental_lead.wav,71,1.385,16
...,...,...,...,...
86,\instrumental_lead.wav,81,48.039,17
87,\instrumental_lead.wav,81,49.269,17
88,\instrumental_lead.wav,83,49.705,19
89,\instrumental_lead.wav,84,50.107,20


# Experiment with df and list

In [10]:
# Read WAV File

audio_files = []
path_wav = "audiofile"
path_xml = "annotation"

# Init dict
ref_note = {}
note = 0
l_r_n = []

for filename in sorted(glob.glob(os.path.join(path_xml, '*.xml'))):
    
    # Read data from xml
    obj = untangle.parse(filename)
    audiofile = obj.instrumentRecording.globalParameter.audioFileName.cdata
    pitch = obj.instrumentRecording.transcription.event.pitch.cdata
    #onsetSec = obj.instrumentRecording.transcription.event.onsetSec.cdata
    #offsetSec = obj.instrumentRecording.transcription.event.offsetSec.cdata
    fretNumber = obj.instrumentRecording.transcription.event.fretNumber.cdata
    stringNumber = obj.instrumentRecording.transcription.event.stringNumber.cdata
    
    # Read wav
    wav_file = path_wav + '/' + audiofile
    rate, data = wav.read(wav_file)
    
    # Snip and downsampling audio-file
    data, rate = snip_wav(data, rate, start_sec, end_sec)
    data, rate = easy_downsampling(data, rate, sampl_fac)
    
    # Calc FFT
    #freq, fourier = fourier_trans(data, rate, float(onsetSec), float(offsetSec))
    #freq, fourier = fourier_trans(data, rate, 0, 2.5)
    #freq, fourier = fourier_trans(data, rate, 0, end_sec-start_sec)
    freq, fourier = new_fft(data, rate)
    
    # Update Dict
    #ref_note[filename] = {
    ref_note = {
        "pitch": int(pitch),
        "fret": int(fretNumber),
        "string": int(stringNumber),
        "rate": int(rate),
        "data": data,
        #"onset": float(onsetSec), 
        #"offset": float(offsetSec), 
        "freq": freq,
        "fourier": fourier,
    }
    l_r_n.append(ref_note)
    #note+=1

In [11]:
#list_note = [ref_note]
#list_note
l_r_n

[{'pitch': 40,
  'fret': 0,
  'string': 1,
  'rate': 11025,
  'data': [-415,
   -504,
   -610,
   -731,
   -865,
   -1015,
   -1170,
   -1322,
   -1436,
   -1508,
   -1531,
   -1486,
   -1402,
   -1269,
   -1102,
   -914,
   -716,
   -510,
   -287,
   -47,
   211,
   490,
   793,
   1101,
   1389,
   1652,
   1845,
   1960,
   1984,
   1905,
   1739,
   1515,
   1240,
   957,
   700,
   482,
   322,
   227,
   183,
   194,
   236,
   288,
   349,
   385,
   415,
   439,
   461,
   494,
   538,
   603,
   681,
   748,
   800,
   822,
   797,
   733,
   625,
   468,
   301,
   128,
   -34,
   -146,
   -215,
   -239,
   -205,
   -143,
   -57,
   47,
   133,
   188,
   193,
   140,
   28,
   -142,
   -354,
   -586,
   -841,
   -1087,
   -1323,
   -1543,
   -1755,
   -1957,
   -2163,
   -2355,
   -2565,
   -2780,
   -2964,
   -3124,
   -3239,
   -3283,
   -3245,
   -3127,
   -2916,
   -2640,
   -2318,
   -1974,
   -1634,
   -1318,
   -1030,
   -785,
   -554,
   -325,
   -84,
   185,
   491,

In [12]:
df_ = pd.DataFrame(l_r_n)
df_

,pitch,fret,string,rate,data,freq,fourier
0,40,0,1,11025,"[-415, -504, -610, -731, -865, -1015, -1170, -...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[1.270891280686903, 1.1001212440002928, 1.0035..."
1,41,1,1,11025,"[1522, 1467, 1389, 1285, 1147, 976, 770, 532, ...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[2.6218406095053814, 2.7138286312368556, 2.573..."
2,42,2,1,11025,"[1294, 1152, 986, 799, 601, 391, 177, -37, -25...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[2.0164469706131336, 2.0133184082902593, 1.902..."
3,43,3,1,11025,"[1316, 1021, 706, 381, 72, -201, -430, -606, -...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[0.06869028903132181, 0.12035485471778247, 0.1..."
4,44,4,1,11025,"[-995, -1203, -1422, -1636, -1838, -2006, -212...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[3.373322046196638, 3.4391332149224807, 3.0520..."
...,...,...,...,...,...,...,...
73,72,8,6,11025,"[-1506, -795, -200, 219, 988, 1698, 2090, 2077...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[0.4672874591849075, 0.5666033683419208, 1.485..."
74,73,9,6,11025,"[-775, -1109, -1372, -1633, -1776, -1864, -150...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[0.9954045229169186, 0.9417963453390442, 0.544..."
75,74,10,6,11025,"[1410, 1604, 1622, 1647, 1472, 950, 310, -195,...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[0.43209577941710003, 0.881355986934138, 0.331..."
76,75,11,6,11025,"[1053, 1603, 1716, 1653, 1511, 1309, 1023, 546...","[0.0, 0.6667271407837445, 1.333454281567489, 2...","[0.7061313338976902, 1.0465492312111073, 1.031..."


In [13]:
columns = ['pitch', 'freq', 'fourier']
df__ = df_[columns]
df__.loc[df__['pitch'] == 45]


,pitch,freq,fourier
5,45,"[0.0, 0.6667271407837445, 1.333454281567489, 2...","[2.1713629217559562, 2.2941214208439766, 2.600..."
6,45,"[0.0, 0.6667271407837445, 1.333454281567489, 2...","[1.3352279598500423, 1.3055424400361224, 1.725..."


In [15]:
#df__.groupby(['pitch']).agg(
#mean_freq = pd.NamedAgg(column='freq', aggfunc=sum),
#mean_fourier = pd.NamedAgg(column='fourier', aggfunc=sum))
grouped_df = df__.groupby(['pitch']).sum().reset_index()
grouped_df.loc[grouped_df['pitch'] == 45]


SyntaxError: invalid syntax (<ipython-input-15-cb912a148354>, line 5)

# Read audiofile for approximation

In [ ]:
# Chords
#rate_chord, data_chord = wav.read('audiofile/1-E1-Major 00.wav')
#rate_chord, data_chord = wav.read('audiofile/1-E1-Major 01.wav')

# Single note
#rate_chord, data_chord = wav.read('audiofile/G53-65601-1111-00067.wav')
rate_chord, data_chord = wav.read('audiofile/G53-40100-1111-00001.wav')

# Single note played on another guitar
#rate_chord, data_chord = wav.read('audiofile/G53-42102-1111-237.wav')

In [ ]:
# mischsignal mischsignal
data_20, rate_20 = snip_wav(data_chord, rate_chord, start_sec, end_sec)
data_ds, rate_ds = easy_downsampling(data_20, rate_20, sampl_fac)
freq, mischsignal = new_fft(data_ds, rate_ds)
#freq, mischsignal = new_fft(data_chord, rate_chord)

plot_freq_domain(freq, mischsignal, 'Mischsignal')
#print('data:', len(data_ds))
#print('rate:', rate_ds)
print('Länge Mischsignal: ', len(mischsignal))

# Gurobi

In [ ]:
# Build Model
m = Model('AMt')

lambs = []
for lamb in range(len(ref_note)):
    lambs.append(m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'lamb_' + str(lamb)))
m.update()

In [ ]:
'''
%%time
#Optimize Signal

deviation = 0
approxi_sgn = 0
 
for sgn in range(len(mischsignal)):
#for sgn in range(100):
    for lamb in range(len(lambs)):
        approxi_sgn += lambs[lamb] * ref_note[lamb]['fourier'][sgn]
        
    deviation += ((mischsignal[sgn] - approxi_sgn)*(mischsignal[sgn] - approxi_sgn))
    # deviation wird ausmultipliziert stattdessen die für 'mischsignal[sgn] - approxi_sgn'
    # eine Gurobi-Variable (m.addVar  inkl. constr.) verwenden
    # Überprüfen ob der Betrag der Differenz besser geeignet ist?
print('Deviation is calculated.')
'''

In [ ]:
%%time
# Schlaufen vertauscht
deviation = 0
approxi_sgn = 0

for lamb in range(len(lambs)):
    la = lambs[lamb]
    print('la:', la)
    note = ref_note[lamb]['fourier']
    
    for sgn in range(len(mischsignal)):
        approxi_sgn = la * note[sgn]
        mi = mischsignal[sgn]
        deviation += ((mi - approxi_sgn)*(mi - approxi_sgn))
print('Deviation is calculated.')

In [ ]:
%%time
m.setObjective(deviation, GRB.MINIMIZE)
m.optimize()

approx_v = []

for v in m.getVars():
    print('%s: %g' % (v.varName, v.x))
    approx_v.append(v.x)

# Metrik berechnen

In [ ]:
target_v = np.zeros(78,dtype=int)
target_v[0] = 1

In [ ]:
metric(target_v, approx_v)